# NanoChat Easy - SFT Training


## Import model and tokenizer


In [1]:
import torch
from torch.utils.data import DataLoader
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, get_linear_schedule_with_warmup


model_id = "karpathy/nanochat-d32"
revision = "refs/pr/1"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


tokenizer = AutoTokenizer.from_pretrained(model_id, revision=revision)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    revision=revision,
    torch_dtype=torch.bfloat16 if device.type == "cuda" else torch.float32,
).to(device)


/fsx/benjamin_burtenshaw/nanochat_/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
`torch_dtype` is deprecated! Use `dtype` instead!


## Setup LoRA


In [ ]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=1,
    lora_alpha=2,
    lora_dropout=0.00,
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "fc1", "fc2"]
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


## Demo the model


In [ ]:
print("=" * 80)
print("TEST 1: Plain Autoregressive Prompt")
print("=" * 80)
prompt = "The Eiffel Tower stands in Paris and"
test_inputs = tokenizer(prompt, return_tensors="pt").to(device)


with torch.no_grad():
    test_outputs = model.generate(
        **test_inputs,
        max_new_tokens=64,
        do_sample=False,
        pad_token_id=tokenizer.pad_token_id,
    )

generated_tokens = test_outputs[0, test_inputs["input_ids"].shape[1] :]
print(f"Prompt: {prompt}")
print(f"\nGenerated: {tokenizer.decode(generated_tokens, skip_special_tokens=True)}")
print("=" * 80)


In [ ]:
print("=" * 80)
print("TEST 2: Chat Template")
print("="*80)
conversation = [
    {"role": "user", "content": "What is the capital of France?"},
]

inputs = tokenizer.apply_chat_template(
    conversation, add_generation_prompt=True, tokenize=True, return_dict=True, return_tensors="pt"
).to(device)

print(f"Formatted prompt: {tokenizer.decode(inputs['input_ids'][0])}")
print(f"Input IDs: {inputs['input_ids'][0].tolist()}")

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=64,
        do_sample=False
    )

generated_tokens = outputs[0, inputs["input_ids"].shape[1] :]
print(f"\nGenerated: {tokenizer.decode(generated_tokens)}")
print("=" * 80)


## Dataset


In [ ]:
raw_dataset = load_dataset("HuggingFaceTB/smoltalk2", "SFT", split="OpenThoughts3_1.2M_think")
splits = raw_dataset.train_test_split(test_size=0.1, seed=42)
train_dataset = splits["train"]
eval_dataset = splits["test"]


### Process the Dataset


In [ ]:
max_length = 2048
max_train_examples = None
max_eval_examples = None

def format_example(example):
    formatted = tokenizer.apply_chat_template(
        example["messages"],
        add_generation_prompt=False,
        truncation=True,
        max_length=max_length,
        padding=False,
        return_dict=True,
        return_tensors="pt",
    )
    return {
        "input_ids": formatted["input_ids"][0].tolist(),
        "attention_mask": formatted["attention_mask"][0].tolist(),
    }


if max_train_examples is not None:
    train_dataset = train_dataset.select(range(min(len(train_dataset), max_train_examples)))
    train_dataset = train_dataset.map(format_example, remove_columns=train_dataset.column_names)
else:
    train_dataset = train_dataset.map(format_example, remove_columns=train_dataset.column_names)
    
if max_eval_examples is not None:
    eval_dataset = eval_dataset.select(range(min(len(eval_dataset), max_eval_examples)))
    eval_dataset = eval_dataset.map(format_example, remove_columns=eval_dataset.column_names)
else:
    eval_dataset = eval_dataset.map(format_example, remove_columns=eval_dataset.column_names)


## Training Configuration

In [ ]:
train_batch_size = 2
eval_batch_size = 2
num_epochs = 1
gradient_accumulation_steps = 4
learning_rate = 1e-5
weight_decay = 0.0
warmup_ratio = 0.03
logging_frequency = 10


## Create a `DataLoader` 👴

In [ ]:
def collate_fn(batch):
    batch_dict = {
        "input_ids": [record["input_ids"] for record in batch],
        "attention_mask": [record["attention_mask"] for record in batch],
    }
    padded = tokenizer.pad(batch_dict, padding=True, return_tensors="pt")
    labels = padded["input_ids"].clone()
    labels[padded["attention_mask"] == 0] = -100
    padded["labels"] = labels
    return padded


TrainLoader = DataLoader(train_dataset, batch_size=train_batch_size, shuffle=True, collate_fn=collate_fn)
EvalLoader = DataLoader(eval_dataset, batch_size=eval_batch_size, shuffle=False, collate_fn=collate_fn)


## Optimizer

In [ ]:
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=learning_rate,
    weight_decay=weight_decay,
)


# Learning Rate Scheduler

In [ ]:
num_update_steps_per_epoch = max(len(TrainLoader) // gradient_accumulation_steps, 1)
max_train_steps = num_epochs * num_update_steps_per_epoch
warmup_steps = max(1, int(max_train_steps * warmup_ratio))
scheduler = get_linear_schedule_with_warmup(optimizer, warmup_steps, max_train_steps)


# The Training Loop

In [ ]:

model.train()
global_step = 0
running_loss = 0.0
running_steps = 0

for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")
    optimizer.zero_grad(set_to_none=True)
    for step, batch in enumerate(TrainLoader, start=1):
        batch = {key: value.to(device) for key, value in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss / gradient_accumulation_steps
        loss.backward()

        running_loss += outputs.loss.float().item()
        running_steps += 1

        if step % gradient_accumulation_steps == 0 or step == len(TrainLoader):
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad(set_to_none=True)
            global_step += 1

            if global_step % logging_frequency == 0:
                current_lr = scheduler.get_last_lr()[0]
                mean_loss = running_loss / running_steps
                print(f"step={global_step:05d} | loss={mean_loss:.4f} | lr={current_lr:.2e}")
                running_loss = 0.0
                running_steps = 0

    train_loss = running_loss / running_steps if running_steps > 0 else float("nan")
    print(f"Training loss after epoch {epoch + 1}: {train_loss:.4f}")
    
    model.eval()
    losses = []
    with torch.no_grad():
        for _, batch in enumerate(EvalLoader, start=1):
            batch = {key: value.to(device) for key, value in batch.items()}
            loss = model(**batch).loss
            losses.append(loss.float().item())
    model.train()
    val_loss = sum(losses) / len(losses) if losses else float("nan")
        
    print(f"Validation loss after epoch {epoch + 1}: {val_loss:.4f}")

print("Training complete.")
